# binder-profiling
Read your .csv file and give you better insight of your data using [Pandas Profiling](pandas-profiling). Please consider to visit and leave a star on [binder-profiling](https://github.com/yasirroni/binder-profiling) if you like this repository.

In [1]:
from IPython.display import display
from ipywidgets import widgets
import pandas as pd
import numpy as np
import csv
import io
import time
import threading

from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

In [2]:
# uploader
uploader = widgets.FileUpload(
    accept='.csv*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    )

# label
label = widgets.Label(
    value = 'Upload File in .csv Format'
)

# wrapper
wrapper = widgets.VBox(
    children=(uploader, label)
)

In [3]:
# get content and name
def get_content_and_name(uploader):
    for key, value in uploader.value.items():
        return value['content'], value['metadata']['name']

In [4]:
def write_file_from_bytes_data(bytes_data, file_name='temp.csv'):
    with open(file_name,'wb') as f:
        f.write(content)

In [5]:
def work():
    # global flag
    global thread_status, content, extension, profile
    
    # save uploader.value for state reference
    current_uploader_content = uploader.value
    
    while thread_status:
        
        # check if there is a change in uploader.value
        if current_uploader_content != uploader.value:
            
            # renew state reference
            current_uploader_content = uploader.value
           
            if uploader.value:
                # message
                label.value = 'Reading Your File...'
                
                # get content and file name
                content, file_name = get_content_and_name(uploader)
                
                # from bytes data to csv file
                write_file_from_bytes_data(content, file_name=file_name)
                
                # pandas read csv
                df = pd.read_csv(file_name)
                
                label.value = 'Profiling Your File...'
                
                # generate the Profiling Report
                profile = ProfileReport(df, title=file_name.split('.')[-1], html={'style': {'full_width': True}}, sort="None")

                # The Notebook Widgets Interface
                display(profile)
                
                label.value = 'Profiling Finished!'
        
        # sleep for threading
        time.sleep(0.05)
        
        # exit flag from threading
        if thread_status == False:
            break

In [6]:
# thread status
thread_status = False
thread_status = True

# create thread
thread = threading.Thread(target=work)

# start thread
thread.start()

In [7]:
display(wrapper)